In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import os
import pandas as pd 
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Conv2DTranspose, UpSampling2D, Concatenate
from tensorflow.keras.optimizers import RMSprop,Adam

Loading data

In [ ]:
training_images = []
training_groundTruth = []
validation_images = []
validation_groundTruth = []
test_images = []
test_groundTruth = []

path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/train/images/images"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  training_images.append(img)
    
path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/train/masks/masks"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  training_groundTruth.append(img)
    
path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/valid/images/images"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  validation_images.append(img)
    
path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/valid/masks/masks"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  validation_groundTruth.append(img)
    
path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/test/images/images"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  test_images.append(img)
    
path = "../input/isic2017-256x256-jpeg/ISIC_2017_256x256/test/masks/masks"
paths = os.listdir(path)
paths.sort()
for image in paths:
  img = cv2.imread(path + '/' + image)
  test_groundTruth.append(img)

In [ ]:
training_images = np.array(training_images)
training_groundTruth = np.array(training_groundTruth)
validation_images = np.array(validation_images)
validation_groundTruth = np.array(validation_groundTruth)
test_images = np.array(test_images)
test_groundTruth = np.array(test_groundTruth)
training_images.sort()
training_groundTruth.sort()
input_shape = training_images[0].shape
print(input_shape)
print(training_groundTruth.shape)

Image samples

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(cv2.cvtColor(training_images[0], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,2)
plt.imshow(cv2.cvtColor(training_images[55], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,3)
plt.imshow(cv2.cvtColor(training_images[130], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,4)
plt.imshow(cv2.cvtColor(training_images[255], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,5)
plt.imshow(cv2.cvtColor(training_images[512], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,6)
plt.imshow(cv2.cvtColor(training_images[1024], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,7)
plt.imshow(cv2.cvtColor(training_images[1350], cv2.COLOR_BGR2RGB))
plt.subplot(2,4,8)
plt.imshow(cv2.cvtColor(training_images[1890], cv2.COLOR_BGR2RGB))
plt.show()

Image masks

In [ ]:
plt.figure(figsize=(20,9))
plt.subplot(2,4,1)
plt.imshow(training_groundTruth[0], cmap = plt.cm.binary_r)
plt.subplot(2,4,2)
plt.imshow(training_groundTruth[55], cmap = plt.cm.binary_r)
plt.subplot(2,4,3)
plt.imshow(training_groundTruth[130], cmap = plt.cm.binary_r)
plt.subplot(2,4,4)
plt.imshow(training_groundTruth[255], cmap = plt.cm.binary_r)
plt.subplot(2,4,5)
plt.imshow(training_groundTruth[512], cmap = plt.cm.binary_r)
plt.subplot(2,4,6)
plt.imshow(training_groundTruth[1024], cmap = plt.cm.binary_r)
plt.subplot(2,4,7)
plt.imshow(training_groundTruth[1350], cmap = plt.cm.binary_r)
plt.subplot(2,4,8)
plt.imshow(training_groundTruth[1890], cmap = plt.cm.binary_r)
plt.show()

Unet architecture

In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(filters=3, kernel_size=(1,1), padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_unet(input_shape)
    model.summary()


Loss and metrics

In [ ]:
from keras import backend as K

# def IoU(y_true, y_pred, smooth=1):
#     y_true = K.flatten(y_true)
#     y_pred = K.flatten(y_pred)
#     intersection = K.sum(y_true * y_pred)
#     total = K.sum(y_true + y_pred)
#     union = total - intersection
#     IoU = (intersection + smooth)/(union + smooth)
#     return  IoU

def IoU(y_true, y_pred, smooth=1):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    total = K.sum(y_true + y_pred)
    union = total - intersection
    IoU = (intersection + smooth)/(union + smooth)
    return  IoU

def jaccard_similarity(y_true, y_pred, smooth=1):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
#     y_true = K.flatten(y_true)
#     y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    total = K.sum(y_true + y_pred)
    union = total - intersection
    IoU = (intersection + smooth)/(union + smooth)
    return 1 - IoU

def jacc_loss(y_true, y_pred):
     return 1-IoU(y_true, y_pred)

Training network

In [ ]:
model.compile(loss=[jaccard_similarity],
                   optimizer=Adam(learning_rate=1e-4),
                   metrics=[IoU])

result = model.fit(training_images,training_groundTruth.astype(np.float32),
                    validation_data=(validation_images, validation_groundTruth.astype(np.float32)),
                    batch_size=24,
                    epochs=5,
                    verbose=1)